In [1]:
import pandas as pd
import numpy as np
# Relative path from notebooks/ to data/
file_path = "../data/google_stock_data.xlsx"

# Read the Excel file
df = pd.read_excel(file_path)

# Round all numerical columns to 2 decimal places
df = df.round(2)

# update the date format to YYYY-MM-DD
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

In [2]:
# Rename columns
df.rename(columns={'Close_GOOG': 'Close', 'High_GOOG': 'High'
                   ,'Low_GOOG': 'Low', 'Open_GOOG': 'Open'
                   ,'Volume_GOOG': 'Volume'}, inplace=True)

# Display the columns of the DataFrame
df.head()

,Date,Close,High,Low,Open,Volume
0,2020-01-02,67.90,67.94,66.62,66.62,28132000
1,2020-01-03,67.57,68.16,66.82,66.93,23728000
2,2020-01-06,69.24,69.35,67.04,67.04,34646000
3,2020-01-07,69.19,69.67,69.05,69.42,30054000
4,2020-01-08,69.74,70.10,69.07,69.13,30560000


1. Group by Month and calculate:

- mean Close

- max High

- min Low

In [3]:
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

# Apply aggregate functions
agg_df = df.groupby("Month").agg({'Close' : 'mean',
                        'High' : 'max',
                        'Low' : 'min'})

# Rename columns
agg_df.rename(columns={'Close': 'Mean_close', 'High': 'Max_high'
                   ,'Low': 'Min_low'}, inplace=True)

agg_df

,Mean_close,Max_high,Min_low
Month,,,
1,107.006040,154.15,66.62
2,110.406458,151.07,63.12
3,106.708739,152.16,50.33
4,113.640680,175.22,53.62
5,118.284095,178.73,64.51
6,120.047619,186.44,66.89
7,124.807905,192.21,70.01
8,125.622613,174.68,72.44
9,122.057961,166.63,69.85


2. Extract month from Date and compute sum of Volume.

In [4]:
df['Month'] = df['Date'].dt.to_period('M')
df.groupby('Month')['Volume'].sum()

Month
2020-01     673832000
2020-02     741612000
2020-03    1427842000
2020-04     926702000
2020-05     637326000
2020-06     776568000
2020-07     704940000
2020-08     756618000
2020-09     811352000
2020-10     785084000
2020-11     700708000
2020-12     629318000
2021-01     661654000
2021-02     588580000
2021-03     695884000
2021-04     614494000
2021-05     509114000
2021-06     548656000
2021-07     456776000
2021-08     372312000
2021-09     505198000
2021-10     496572000
2021-11     443172000
2021-12     479260000
2022-01     615510000
2022-02     732650000
2022-03     637420000
2022-04     564940000
2022-05     717486000
2022-06     627132000
2022-07     638766600
2022-08     430961500
2022-09     532993100
2022-10     584537500
2022-11     603449000
2022-12     498585400
2023-01     526178100
2023-02     754400500
2023-03     725477100
2023-04     461670700
2023-05     620361600
2023-06     521440000
2023-07     525480300
2023-08     463504700
2023-09     389639800
2023

3. Count number of days Close price was above its daily Open. Also count days when Close price below the Open price.

In [5]:
# days when Close > Open
df['Close_high'] = np.where(df['Close'] > df['Open'], True, False)
df['Close_high'].sum()

689

In [6]:
# days when Close < Open
false_cnt = (~df.Close_high).sum()
false_cnt

569

4. Identify the day with the highest % price increase

In [7]:
df['percent_change'] = ((df['Close'] - df['Open'])/df['Open'])*100

df.loc[df['percent_change'].idxmax()]['Date']

Timestamp('2022-11-30 00:00:00')

Multi Level Groupby

5. Compute weekly:

- average Close

- total Volume

- highest High

In [8]:
# Create year and week number columns
df['iso_year'] = df['Date'].dt.isocalendar().year
df['week_number_iso'] = df['Date'].dt.isocalendar().week

# Apply aggregate functions
weekly_agg_df = df.groupby(["iso_year","week_number_iso"]).agg({'Close' : 'mean',
                        'Volume' : 'sum',
                        'High' : 'max'})

# Rename columns
weekly_agg_df.rename(columns={'Close': 'Mean_close', 'Volume': 'Total_Volume'
                   ,'High': 'Max_high'}, inplace=True)

weekly_agg_df

Mean_close  Total_Volume  Max_high
iso_year week_number_iso                                    
2020     1                    67.735      51860000     68.16
         2                    69.936     161692000     71.26
         3                    71.922     161276000     73.56
         4                    73.545     135666000     74.65
         5                    71.860     163338000     73.01
...                              ...           ...       ...
2024     49                  173.818      80019700    176.94
         50                  188.460     139950700    196.25
         51                  192.994     156317400    202.23
         52                  195.540      44646600    197.52
2025     1                   190.950      26564700    193.15

[262 rows x 3 columns]

6. For each month, find the day with the highest Volume

In [9]:
idx = df.groupby('Month')['Volume'].idxmax()
df.loc[idx]


,Date,Close,High,Low,Open,Volume,Month,Close_high,percent_change,iso_year,week_number_iso
20,2020-01-31,71.22,73.01,70.94,72.95,48344000,2020-01,False,-2.371487,2020,5
22,2020-02-04,71.86,72.98,70.83,72.36,78660000,2020-02,False,-0.690989,2020,6
50,2020-03-16,53.85,57.22,53.36,54.43,85048000,2020-03,False,-1.065589,2020,12
81,2020-04-29,66.62,67.54,65.82,66.62,75872000,2020-04,False,0.000000,2020,18
83,2020-05-01,65.58,67.14,65.10,65.97,41450000,2020-05,False,-0.591178,2020,18
122,2020-06-26,67.53,71.19,67.14,71.08,85354000,2020-06,False,-4.994373,2020,26
146,2020-07-31,73.64,74.93,72.21,74.74,68798000,2020-07,False,-1.471769,2020,31
164,2020-08-26,82.06,82.40,79.64,79.85,79868000,2020-08,True,2.767689,2020,35
170,2020-09-03,81.53,84.90,80.20,84.90,62156000,2020-09,False,-3.969376,2020,36
210,2020-10-30,80.50,83.78,79.68,83.04,86582000,2020-10,False,-3.058767,2020,44


7. Create a new column: Rolling 7-day average Close

In [10]:
df['Close'].rolling(7).mean()

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
           ...    
1253    193.894286
1254    193.742857
1255    193.304286
1256    193.665714
1257    193.771429
Name: Close, Length: 1258, dtype: float64

### Window Functions

8. Compute day-to-day change in volume.

In [11]:
df['Volume_diff'] = df['Volume'] - df['Volume'].shift(1)
df

,Date,Close,High,Low,Open,Volume,Month,Close_high,percent_change,iso_year,week_number_iso,Volume_diff
0,2020-01-02,67.90,67.94,66.62,66.62,28132000,2020-01,True,1.921345,2020,1,NaN
1,2020-01-03,67.57,68.16,66.82,66.93,23728000,2020-01,True,0.956223,2020,1,-4404000.0
2,2020-01-06,69.24,69.35,67.04,67.04,34646000,2020-01,True,3.281623,2020,2,10918000.0
3,2020-01-07,69.19,69.67,69.05,69.42,30054000,2020-01,False,-0.331317,2020,2,-4592000.0
4,2020-01-08,69.74,70.10,69.07,69.13,30560000,2020-01,True,0.882395,2020,2,506000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1253,2024-12-24,196.93,197.03,194.57,195.54,6809800,2024-12,True,0.710852,2024,52,-8426100.0
1254,2024-12-26,196.46,197.52,195.24,196.10,7907900,2024-12,True,0.183580,2024,52,1098100.0
1255,2024-12-27,193.41,196.16,191.35,195.84,14693000,2024-12,False,-1.240809,2024,52,6785100.0
1256,2024-12-30,192.07,193.15,189.75,190.25,12209500,2024-12,True,0.956636,2025,1,-2483500.0


9. Compute a 3-day moving high price maximum

In [15]:
df['High_3day_max'] = df['High'].rolling(3).max()
df[['High', 'High_3day_max']]

,High,High_3day_max
0,67.94,NaN
1,68.16,NaN
2,69.35,69.35
3,69.67,69.67
4,70.10,70.10
...,...,...
1253,197.03,197.03
1254,197.52,197.52
1255,196.16,197.52
1256,193.15,197.52


10. Create a column that ranks days by Close price within each month.

In [16]:
df.groupby('Month')['Close'].rank(method='dense', ascending=False)

0       19.0
1       20.0
2       17.0
3       18.0
4       16.0
        ... 
1253     2.0
1254     4.0
1255     7.0
1256    10.0
1257    12.0
Name: Close, Length: 1258, dtype: float64